<h1>Content</h1>

1. Connecting to database and exploring the data
2. Research
3. Conclusions

# Retrieve Bookstore Data Using SQL

Our company bought a large service for reading books by subscription. The task is to retrieve data from a database and perform the analysis.
The database contains information about books, publishers, authors, as well as user reviews of books.

**Analysis plan**

- Calculate how many books came out after January 1, 2000;
- For each book, calculate the number of reviews and the average rating;
- Determine the publishing house that has released the largest number of books thicker than 50 pages - so we exclude brochures from the analysis;
- Determine the author with the highest average rating of books - we will take into account only books with 50 or more ratings;
- Calculate the average number of reviews from users who have given more than 50 ratings.

**Data Description**

<u>Table books</u> - contains data about books:

- book_id — book ID;
- author_id — author ID;
- title — book title;
- num_pages — number of pages;
- publication_date — book publication date;
- publisher_id — publisher ID.

<u>Table authors</u> - contains information about the authors:

- author_id — author ID;
- author — author's name.

<u>Table publishers</u> - contains information about publishers:

- publisher_id — publisher identifier;
- publisher — publisher name;

<u>Table ratings</u> - contains data about user ratings of books:

- rating_id — rating ID;
- book_id — book ID;
- username — the name of the user who left the rating;
- rating — book rating.

<u>Table reviews</u> - contains data about user reviews of books:

- review_id — review ID;
- book_id — book ID;
- username — the name of the user who wrote the review;
- text — review text.
<br><br>

**ERD**

<img src="https://www.notion.so/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&table=block&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=e61b3059-2e3b-4865-a7aa-189696da4398&cache=v2"  />

## Connecting to database and exploring the data

Import libraries, setting options:

In [1]:
# Import libraries
import pandas as pd
from sqlalchemy import create_engine

# setting options
db_config = {'user': 'praktikum_student', # user's name
 'pwd': 'Sdf4$2;d-d30pp', # password
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # connection port
 'db': 'data-analyst-final-project-db'} # database name
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# save the connector
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Let's explore the data by displaying the first 5 rows of each table:

In [2]:
for i in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print('Table ', i)
    display(pd.io.sql.read_sql(f'''SELECT * FROM {i} LIMIT 5''', con = engine))
    print()
    print()

Table  books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268




Table  authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd




Table  publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company




Table  ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2




Table  reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Now we understand what data we have.

## Queries

1. Calculate how many books were published after January 1, 2000:

In [3]:
query = '''
SELECT
    COUNT(book_id)
FROM
    books
WHERE
    publication_date > '2000-01-01'
'''

In [4]:
pd.io.sql.read_sql(query, con = engine)

,count
0,819


Since January 1, 2000, 819 books have been published.
<br><br>

2. For each book, calculate the number of reviews and the average rating:

In [5]:
query = '''
WITH a AS (
SELECT
    books.book_id,
    books.title,
    COUNT(reviews.review_id) as count_reviews
FROM
    books
FULL JOIN reviews ON  reviews.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    count_reviews DESC
    ),
b as (
SELECT
    books.book_id,
    books.title,
    AVG(ratings.rating) as avg_rating
FROM
    books
FULL JOIN ratings ON  ratings.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    avg_rating DESC
    )
SELECT
    a.book_id,
    a.title,
    a.count_reviews,
    b.avg_rating
FROM
    a
FULL JOIN b ON  a.book_id = b.book_id
ORDER BY
    count_reviews DESC,
    avg_rating DESC
LIMIT 10
'''

In [6]:
pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


The table above shows the top 10 books by number of reviews. In first place is the book "Twilight (Twilight #1)" with an average rating of 3.66. There are no books in the table with less than 6 reviews. The book with the highest rating in the top is Harry Potter and the Prisoner of Azkaban, its rating is 4.4.
<br><br>

3. Determine a publishing house that has released the largest number of books with more than 50 pages (this condition allows us to exclude brochures).

In [7]:
query = '''
SELECT
    publisher,
    COUNT(book_id) as count_book
FROM
    publishers
FULL JOIN books ON books.publisher_id = publishers.publisher_id 
WHERE 
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    count_book DESC
LIMIT 10
'''

In [8]:
pd.io.sql.read_sql(query, con = engine)

,publisher,count_book
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


The largest number of books was published by Penguin Books.
<br><br>

4. Find the author with the highest average book rating. Consider only books with 50 or more ratings:

In [9]:
query = '''
WITH bookdata as (
SELECT 
    authors.author,
    ratings.book_id,
    COUNT(ratings.rating) as count_rating,
    AVG(ratings.rating) as avg_rating
FROM 
    ratings
LEFT JOIN books ON ratings.book_id = books.book_id
LEFT JOIN authors ON books.author_id = authors.author_id
GROUP BY
    authors.author,
    ratings.book_id
HAVING
   COUNT(rating) > 50
   )

SELECT 
    author,
    AVG(avg_rating) as avg_rating
FROM 
    bookdata
GROUP BY
    author
ORDER BY
    avg_rating DESC
LIMIT 10
'''

In [10]:
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


The author with the highest average book rating is J.K. Rowling/Mary GrandPre. The average rating for her books is 4.3.
<br><br>

5.Calculate the average number of reviews from users who have given more than 50 ratings.

In [11]:
query = '''
SELECT
 AVG (number_of_reviews) AS AVG_count
 FROM (
   SELECT COUNT (review_id) AS number_of_reviews
   FROM reviews
   INNER JOIN (SELECT username,
                      COUNT (rating_id) AS number_of_ratings
               FROM ratings
               GROUP BY username
               HAVING COUNT (rating_id) > 50) AS ratings_users
               ON ratings_users.username = reviews.username
    GROUP BY reviews.username) AS number_of_reviews
'''

In [12]:
pd.io.sql.read_sql(query, con = engine)

,avg_count
0,24.333333


The average number of reviews from users with more than 50 ratings is 24.
<br><br>

## Summary

As a result of queries in the database:

- Since January 1, 2000, 819 books have been published.
- The book "Twilight (Twilight #1)" has the maximum number of user reviews and has an average rating of 3.7 points.
- Penguin Books published the maximum number of books (42 books).
- The author with the highest average book rating (4.3) is J.K. Rowling/Mary GrandPre.
- On average, a user who has put more than 50 ratings writes 24 reviews.